<a href="https://colab.research.google.com/github/ufbfung/fire/blob/main/fire_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
def calculate_fire_number(annual_expenses):
    return annual_expenses * 25

def calculate_future_value(monthly_savings, years):
    months = years * 12
    return monthly_savings * months

def calculate_future_value_compound_growth(principal, interest_rate, years):
    # Convert interest rate to a decimal
    interest_rate_decimal = interest_rate / 100

    # Calculate future value using the compound growth formula
    future_value = principal * (1 + interest_rate_decimal) ** years

    return future_value

def calc_future_value_principal(principal, interest_rate, years):
    interest_rate_decimal = interest_rate / 100
    future_value = principal * (1 + interest_rate_decimal) ** years
    return future_value

def calc_future_value_contributions(monthly_contribution, annual_interest_rate, years):
    # Convert annual interest rate to a decimal
    interest_rate_decimal = annual_interest_rate / 100

    # Calculate pmt for a year
    pmt = monthly_contribution * 12

    # Calculate the future value using the provided formula
    future_value = pmt * ((1 + interest_rate_decimal) ** years - 1) / interest_rate_decimal

    return future_value

def calc_tax_advantaged_contributions(annual_contribution, interest_rate, years):
    # Convert annual interest rate to a decimal
    interest_rate_decimal = interest_rate / 100

    # Calculate pmt for a year
    pmt = annual_contribution

    # Calculate the future value using the provided formula
    future_value = pmt * ((1 + interest_rate_decimal) ** years - 1) / interest_rate_decimal

    return future_value

def calc_future_value_combined(principal, monthly_contribution, saving_years, interest_rate):
    # Calculate the future value with compounding on the principal
    fv_principal = calc_future_value_principal(principal, interest_rate, saving_years)

    # Calculate the future value contributed by monthly contributions
    fv_monthly = calc_future_value_contributions(monthly_contribution, interest_rate, saving_years)

    # Add the future value of the principal and contributions
    fv_total = fv_principal + fv_monthly

    return fv_principal, fv_monthly, fv_total

def calculate_saving_years(current_age, retirement_age):
    saving_years = retirement_age - current_age
    return saving_years

def calculate_savings_needed(fire_number, total_savings, saving_years):
    # Calculate the savings needed
    remaining_savings_needed = fire_number - total_savings

    print(f"\nBased on projected value...\n   You need ${remaining_savings_needed:,.2f} more to hit your FIRE number.")

    # Calculate annual and monthly savings needed based on the remaining savings goal
    annual_savings = remaining_savings_needed / saving_years
    monthly_savings = annual_savings / 12

    return annual_savings, monthly_savings

def calc_401k_ira_hsa_contributions(interest_rate, saving_years):
    # If the user wants to set the max for all three accounts
    set_max = input("Do you want to set the max for 401k, IRA, and HSA? (yes/no): ").lower() == "yes"
    employer_match = float(input("Enter total annual amount of employer match:"))

    if set_max:
        max_401k = 23000
        max_ira = 7000
        max_hsa = 4150
    else:
        max_401k = float(input("Enter the max 401k contribution: "))
        max_ira = float(input("Enter the max IRA contribution: "))
        max_hsa = float(input("Enter the max HSA contribution: "))

    # Calculate the total future value across all three accounts
    total_401k = calc_tax_advantaged_contributions(max_401k + employer_match, interest_rate, saving_years)
    total_ira = calc_tax_advantaged_contributions(max_ira, interest_rate, saving_years)
    total_hsa = calc_tax_advantaged_contributions(max_hsa, interest_rate, saving_years)

    total_contributions = total_401k + total_ira + total_hsa

    return total_contributions, total_401k, total_ira, total_hsa

def main():
    # Get FIRE Goals
    while True:
        try:
            current_age = float(input("Enter your current age: "))
            retirement_age = float(input("Enter your retirement age: "))
            if not (0 <= current_age < 100) or not (0 <= retirement_age < 100):
                raise ValueError("Age values must be between 0 and 100.")
            break  # Exit the loop if both inputs are valid floats
        except ValueError as e:
            print(f"Error: {e}")

    # Get current savings
    while True:
        try:
            current_savings = float(input("Enter your current savings: "))
            break  # Exit the loop if the input is a valid float
        except ValueError as e:
            print(f"Error: {e}")

    # Calculate future value with existing monthly contribution
    monthly_contribution = float(input("Enter your monthly contribution to taxable accounts: "))

    # Calculate saving years
    saving_years = calculate_saving_years(current_age, retirement_age)

    # Set an interest rate
    interest_rate = float(input("Enter interest rate to use for investments: "))

    # Calculate FIRE number
    annual_expenses = float(input("Enter your annual expenses: "))
    fire_number = calculate_fire_number(annual_expenses)

    # Calculate 401k, IRA, HSA contributions
    total_contributions, total_401k, total_ira, total_hsa = calc_401k_ira_hsa_contributions(interest_rate, saving_years)

    # Display results based on assumptions
    print(f"\nYour FIRE number is approximately ${fire_number:,.2f}.")

    # Calculate current gap based on current savings
    gap = fire_number - current_savings
    print(f"Based on current savings, you are currently ${gap:,.2f} away from your FIRE number.")

    # Calculate value total with principal and monthly
    fv_principal, fv_monthly, fv_total = calc_future_value_combined(current_savings, monthly_contribution, saving_years, interest_rate)
    print(f"\nIn {saving_years:,.0f} years...")
    print(f"   Projected value of your principal will be ${fv_principal:,.2f}")
    print(f"   Projected value of your monthly contributions will be ${fv_monthly:,.2f}")
    print(f"   Projected value of both will be ${fv_total:,.2f}")

    # 401k, IRA, HSA contributions
    print(f"\n   Total contributions across 401k, IRA, and HSA will be ${total_contributions:,.2f}")
    print(f"      401k contribution: ${total_401k:,.2f}")
    print(f"      IRA contribution: ${total_ira:,.2f}")
    print(f"      HSA contribution: ${total_hsa:,.2f}")

    # Calculate savings needed
    total_savings = fv_total + total_contributions
    annual_savings, monthly_savings = calculate_savings_needed(fire_number, total_savings, saving_years)
    print(f"   This translates to an additional ${annual_savings:,.2f} annually or ${monthly_savings:,.2f} monthly to hit your FIRE number in {saving_years:,.0f} years.")

if __name__ == "__main__":
    main()

Enter your current age: 36
Enter your retirement age: 40
Enter your current savings: 679000
Enter your monthly contribution to taxable accounts: 3600
Enter interest rate to use for investments: 7
Enter your annual expenses: 59000
Do you want to set the max for 401k, IRA, and HSA? (yes/no): yes
Enter total annual amount of employer match:11500

Your FIRE number is approximately $1,475,000.00.
Based on current savings, you are currently $796,000.00 away from your FIRE number.

In 4 years...
   Projected value of your principal will be $890,030.49
   Projected value of your monthly contributions will be $191,805.54
   Projected value of both will be $1,081,836.03

   Total contributions across 401k, IRA, and HSA will be $202,683.40
      401k contribution: $153,178.03
      IRA contribution: $31,079.60
      HSA contribution: $18,425.76

Based on projected value...
   You need $190,480.57 more to hit your FIRE number.
   This translates to an additional $47,620.14 annually or $3,968.35 mo